In [1]:
from sklearn.cluster import KMeans
from mysql_model import sql4cluster
import pandas as pd
from time import strftime
import os

In [2]:
def read_data_from_csv(user):
    df = pd.read_csv('./result/user_' + str(user) + '_dailyload.csv', header=0)
    return df

In [3]:
def group_user_dailyload(user):
    df = read_data_from_csv(user=user)
    
    km = sql4cluster(model_name='KMeans')
    km_model = km.load_model_from_pkl()
    input_X = df.iloc[:, 4:100] # recorded power consumption
    group = km_model.predict(input_X)
    df.insert(loc=2, column='Group', value=group)
    
    dailyload2group = pd.DataFrame()

    dailyload2group['Week_id'] = df['Week_id']
    dailyload2group['Group_id'] = group
    dailyload2group['Reporttime'] = strftime('%Y-%m-%d %H:%M')
    dailyload2group.insert(loc=0, column='User_id', value=user)

    dailyload2group.to_csv(r'./result/user_' + str(user) + '_group_relation.csv', index=False)
    

In [4]:
def cluster_users_dailyload(folder_name='.'):
    folder = './' + folder_name + '/'
    for filename in os.listdir(folder):
        if filename.find('_dailyload.csv') == -1:
            continue
        user_id = filename.split('user_')[1]
        user_id = user_id.split('_dailyload')[0]
        group_user_dailyload(user_id)